<a href="https://colab.research.google.com/github/AlexGtier/alexgtier.github.io/blob/main/Convert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Convert

Le module Convert est une bibliothèque pour langage de programmation Python. Il fournit de multiples fonctions permettant notamment convertir les données GNSS manipulées :

*   Conversion temporelle :
    * date et heure, temps du jour (TOD),
    * temps de la semaine (TOW),
    * check année bissextile,
    * temps en degrés/minute/seconde

*   Conversion de référentiel :
    * East / North / Up,
    * Lat / Long / Alt (Geodetic),
    * x / y / z (ECEF)


##LIB

In [ ]:
import numpy as np
import pandas as pd
import time
import warnings
from typing import List, Tuple, Union, Any

##TEMPOREL

###date2tow

In [ ]:
def date2tow(y: int, m: int, d: int,
             h: int, mn: int, s: Union[int, float],
             ls: int) -> Union[int, float]:

    """
    Conversion date (UTC) vers time of week (UTC)
    y : year ; m : month ; d : day ; h : hour ;
    mn : minutes ; s : second ; ls : leap second
    (si ls = 18 --> UTC vers GPS sinon mettre à 0)
    """

    y = int(2000 + y % 2000)
    REF_Y = int(1980)
    nb_y_diff = y - REF_Y
    nb_leap_y = 0
    feb = 28
    sec = h*3600 + mn*60 + s + ls

    checkYear = lambda year : (((year % 4 == 0) and
                                (year % 100 != 0)) or
                               (year % 400 == 0))

    for i in range(REF_Y, y):
        if checkYear(i):
            nb_leap_y += 1

    if checkYear(y):
        feb = 29

    nb_d = nb_leap_y*366 + (nb_y_diff-nb_leap_y)*365

    if m == 1:
        nb_d += d - 6
    elif m == 2:
        nb_d += 31 + d - 6
    elif m == 3:
        nb_d += 31 + feb + d - 6
    elif m == 4:
        nb_d += 31 + feb + 31 + d - 6
    elif m == 5:
        nb_d += (31*2) + feb + (30*1) + d-6
    elif m == 6:
        nb_d += (31*3) + feb + (30*1) + d-6
    elif m == 7:
        nb_d += (31*3) + feb + (30*2) + d-6
    elif m == 8:
        nb_d += (31*4) + feb + (30*2) + d-6
    elif m == 9:
        nb_d += (31*5) + feb + (30*2) + d-6
    elif m == 10:
        nb_d += (31*5) + feb + (30*3) + d-6
    elif m == 11:
        nb_d += (31*6) + feb + (30*3) + d-6
    elif m == 12:
        nb_d += (31*6) + feb + (30*4) + d-6

    # week = round(np.floor(nb_d/7))
    sec_tow = ((nb_d % 7)*86400) + sec
    sec_tow = round(sec_tow, 2)

    return sec_tow

###tow2tod

In [ ]:
def tow2tod(tow: List[Union[int, float]]) -> List[Union[int, float]]:

    """
    Conversion multiple time of week (GPS) vers time of day (UTC)
    """

    tow = tow.astype(float)
    day = int((tow[0] - 18)//86400)
    time = tow - day * 86400 - 18
    T = 1
    a = int(time[0])
    b = int(time[-1])
    time = np.arange(a, b+1, 1/T)  # linspace(a, b, int((b - a + 1*T)/T))

    return time


def tow2tod_(tow: Union[int,float], ls: int=18) -> Union[int,float]:

    """
    Conversion unique time of week (GPS) vers time of day
    (GPS vers UTC si ls (leap second) = 18 sinon ls = 0)
    """

    return tow - ls - int((tow-ls)//86400) * 86400

###time2ind

In [ ]:
def time2ind(time: Union[int, float], ti: Union[int, float],
             tf: Union[int, float]) -> Tuple[int, int]:

    """
    Conversion d'un epoch (time) à l'indice de
    début (ti) et de fin (tf) correspondant
    """

    ind_i = 0
    ind_f = 0

    while time[ind_i] != ti:
        ind_i += 1

    while time[ind_f] != tf:
        ind_f += 1

    return ind_i, ind_f

###unix2tow

In [ ]:
def unix2tow(unix: List[Union[int, float]]) -> Union[int, float]:

    """
    Conversion d'un référence temporelle unix
    en seconde UTC vers TOW UTC
    """

    return date2tow(time.gmtime(unix).tm_year,
                    time.gmtime(unix).tm_mon,
                    time.gmtime(unix).tm_mday,
                    time.gmtime(unix).tm_hour,
                    time.gmtime(unix).tm_min,
                    time.gmtime(unix).tm_sec + (unix - int(unix)), 0)

###check_t

In [ ]:
def check_t(t: Union[int, float]) -> Union[int, float]:

    """
    Contrôle si :
    toe (tow) - heure d'emission (t_tx) < half_week
    half_week = (24h+24h+24h+12h) = 302 400 s
    """

    half_week = 302400
    tt = t

    if t > half_week:
        tt = t - 2 * half_week

    if t < - half_week:
        tt = t + 2 * half_week

    return tt

###tod2time

In [ ]:
def tod2time(t: Union[int, float], ls: int=18) -> Tuple[int, int, Union[int, float]]:

    """
    Conversion TOD (UTC) vers heure (h, min, sec) (GPS) si ls = 18
    """
    t += ls
    return int(t // 3600), int((t % 3600) // 60), round((t % 3600) % 60, 3)

###deg2dms

In [ ]:
def deg2dms(deg: Union[int, float]) -> Union[int, float]:

    """
    Conversion of degrees to degrees, minutes, and seconds.
    The output format (dms format) is: (degrees*100 + minutes + seconds/100)
    Save the sign for later processing.
    """

    neg_arg = False

    if deg < 0:
        # Only positive numbers should be used while splitting into deg/min/sec
        deg = -deg
        neg_arg = True

    # Split degrees minutes and seconds
    int_deg = np.floor(deg)
    decimal = deg - int_deg
    min_part = decimal*60
    min = np.floor(min_part)
    sec_part = min_part - np.floor(min_part)
    sec = sec_part*60

    # Check for overflow
    if sec == 60:
        min = min + 1
        sec = 0

    if min == 60:
        int_deg = int_deg + 1
        min = 0

    # Construct the output
    dmsOutput = int_deg * 100 + min + sec/100

    # Correct the sign
    if neg_arg == True:
        dmsOutput = -dmsOutput
    else:
        pass

    return dmsOutput

##RÉFÉRENTIEL

In [ ]:
wgs84 = {"name": "WGS-84 (1984)", "a": 6378137.0, "b": 6356752.31424518}

###sanitize

In [ ]:
def sanitize(lat: float, deg: bool) -> Tuple[float, Any]:

    """
    Conversion deg2rad avec contrôle de la valeur.
    """

    if deg:
        lat = np.radians(lat)

    try:
        if (abs(lat) > np.pi / 2).any():
            raise ValueError("-pi <= latitude <= pi")
    except (AttributeError, TypeError):
        if abs(lat) > np.pi / 2:
            raise ValueError("-pi <= latitude <= pi")

    return lat

###uvw2enu

In [ ]:
def uvw2enu(u: float, v: float, w: float, lat0: float, lon0: float,
            deg: bool = True) -> Tuple[float, float, float]:

    """
    Parameters -----

    u : float
    v : float
    w : float

    Results -------

    East : float
        target east ENU coordinate (meters)
    North : float
        target north ENU coordinate (meters)
    Up : float
        target up ENU coordinate (meters)
    """
    if deg:
        lat0 = np.radians(lat0)
        lon0 = np.radians(lon0)

    cos_lat0 = np.cos(lat0)
    sin_lat0 = np.sin(lat0)
    cos_lon0 = np.cos(lon0)
    sin_lon0 = np.sin(lon0)

    t = cos_lon0 * u + sin_lon0 * v
    East = -sin_lon0 * u + cos_lon0 * v
    Up = cos_lat0 * t + sin_lat0 * w
    North = -sin_lat0 * t + cos_lat0 * w

    return East, North, Up

### enu2uvw

In [ ]:
def enu2uvw(east, north, up, lat0, lon0,
            deg: bool = True) -> Tuple[float, float, float]:
    """
    Parameters
    ----------

    e1
        target east ENU coordinate (meters)
    n1
        target north ENU coordinate (meters)
    u1
        target up ENU coordinate (meters)

    Results
    -------

    u
    v
    w
    """

    if deg:
        lat0 = np.radians(lat0)
        lon0 = np.radians(lon0)

    t = np.cos(lat0) * up - np.sin(lat0) * north
    w = np.sin(lat0) * up + np.cos(lat0) * north

    u = np.cos(lon0) * t - np.sin(lon0) * east
    v = np.sin(lon0) * t + np.cos(lon0) * east

    return u, v, w

###geodetic2ecef

In [ ]:
def geodetic2ecef(lat: float, lon: float, alt: float,
                  deg: bool = True) -> Tuple[float, float, float]:

    """
    Point transformation from Geodetic of specified ellipsoid
    (default WGS-84) to ECEF.

    Parameters -----

    lat : target geodetic latitude
    lon : target geodetic longitude
    h : target altitude above geodetic ellipsoid (meters)
    ell : Ellipsoid wgs84 (default)
    deg : (optional) degrees input/output  (False: radians in/out)

    Returns -------

    ECEF (Earth centered, Earth fixed)  x,y,z

    x : target x ECEF coordinate (meters)
    y : target y ECEF coordinate (meters)
    z : target z ECEF coordinate (meters)
    """

    lat = sanitize(lat, deg)
    if deg:
        lon = np.radians(lon)

    # radius of curvature of the prime vertical section
    semimajor_axis = 6378137.0
    semiminor_axis = 6356752.31424518

    e2 = (semimajor_axis ** 2 - semiminor_axis ** 2) / semimajor_axis ** 2
    N = semimajor_axis ** 2 / np.sqrt(semimajor_axis ** 2 * np.cos(lat) ** 2
                                   + semiminor_axis ** 2 * np.sin(lat) ** 2)

    # Compute cartesian (geocentric) coordinates given  (curvilinear) geodetic
    # coordinates.
    x = (N + alt) * np.cos(lat) * np.cos(lon)
    y = (N + alt) * np.cos(lat) * np.sin(lon)
    # z = (N * (semiminor_axis / semimajor_axis) ** 2 + alt) * np.sin(lat)
    z = (N * (1 - e2) + alt) * np.sin(lat)

    return x, y, z

### enu2ecef

In [ ]:
def enu2ecef(e1, n1, u1, lat0, lon0, h0,
             deg: bool = True) -> Tuple[float, float, float]:
    """
    ENU to ECEF

    Parameters
    ----------

    e1
        target east ENU coordinate (meters)
    n1
        target north ENU coordinate (meters)
    u1
        target up ENU coordinate (meters)
    lat0
        Observer geodetic latitude
    lon0
        Observer geodetic longitude
    h0
         observer altitude above geodetic ellipsoid (meters)
    ell : Ellipsoid
          wgs84 (default)
    deg : bool, optional
          degrees input/output  (False: radians in/out)


    Results
    -------
    x
        target x ECEF coordinate (meters)
    y
        target y ECEF coordinate (meters)
    z
        target z ECEF coordinate (meters)
    """
    x0, y0, z0 = geodetic2ecef(lat0, lon0, h0, deg=deg)
    dx, dy, dz = enu2uvw(e1, n1, u1, lat0, lon0, deg=deg)

    return x0 + dx, y0 + dy, z0 + dz

### ecef2geodetic

In [ ]:
def ecef2geodetic(x, y, z, deg: bool = True) -> Tuple[float, float, float]:
    """
    convert ECEF (meters) to geodetic coordinates

    Parameters
    ----------
    x
        target x ECEF coordinate (meters)
    y
        target y ECEF coordinate (meters)
    z
        target z ECEF coordinate (meters)
    ell : Ellipsoid
          wgs84 (default)
    deg : bool, optional
          degrees input/output  (False: radians in/out)

    Returns
    -------
    lat
           target geodetic latitude
    lon
           target geodetic longitude
    alt
         target altitude above geodetic ellipsoid (meters)

    based on:
    You, Rey-Jer. (2000). Transformation of Cartesian to Geodetic Coordinates without Iterations.
    Journal of Surveying Engineering. doi: 10.1061/(ASCE)0733-9453
    """

    # radius of curvature of the prime vertical section
    semimajor_axis = ['a'] #6378137.0
    semiminor_axis = wgs84['b'] #6356752.31424518

    try:
        x = asarray(x)
        y = asarray(y)
        z = asarray(z)
    except NameError:
        pass

    r = np.sqrt(x**2 + y**2 + z**2)

    E = np.sqrt(semimajor_axis**2 - semiminor_axis**2)

    # eqn. 4a
    u = np.sqrt(0.5 * (r**2 - E**2) + 0.5 * np.hypot(r**2 - E**2, 2 * E * z))

    Q = np.hypot(x, y)

    huE = np.hypot(u, E)

    # eqn. 4b
    try:
        with warnings.catch_warnings(record=True):
            warnings.simplefilter("error")
            Beta = np.arctan(huE / u * z / np.hypot(x, y))
    except (ArithmeticError, RuntimeWarning):
        if np.isclose(z, 0):
            Beta = 0
        elif z > 0:
            Beta = np.pi / 2
        else:
            Beta = -np.pi / 2

    # eqn. 13
    dBeta = ((semiminor_axis * u - semimajor_axis * huE + E**2) * np.sin(Beta)) / (
        semimajor_axis * huE * 1 / np.cos(Beta) - E**2 * np.cos(Beta)
    )

    Beta += dBeta

    # eqn. 4c
    # %% final output
    lat = np.arctan(semimajor_axis / semiminor_axis * np.tan(Beta))

    try:
        # patch latitude for float32 precision loss
        lim_pi2 = np.pi / 2 - np.finfo(dBeta.dtype).eps
        lat = np.where(Beta >= lim_pi2, pi / 2, lat)
        lat = np.where(Beta <= -lim_pi2, -pi / 2, lat)
    except NameError:
        pass

    lon = np.arctan2(y, x)

    # eqn. 7
    cosBeta = np.cos(Beta)
    try:
        # patch altitude for float32 precision loss
        cosBeta = np.where(Beta >= lim_pi2, 0, cosBeta)
        cosBeta = np.where(Beta <= -lim_pi2, 0, cosBeta)
    except NameError:
        pass

    alt = np.hypot(z - semiminor_axis * np.sin(Beta), Q - semimajor_axis * cosBeta)

    # inside ellipsoid?
    inside = (
        x**2 / semimajor_axis**2
        + y**2 / semimajor_axis**2
        + z**2 / semiminor_axis**2
        < 1
    )

    try:
        if inside.any():
            # avoid all false assignment bug
            alt[inside] = -alt[inside]
    except (TypeError, AttributeError):
        if inside:
            alt = -alt

    if deg:
        lat = np.degrees(lat)
        lon = np.degrees(lon)

    return lat, lon, alt

###enu2geodetic

In [ ]:
def enu2geodetic(e, n, u, lat0, lon0, h0,
                 deg: bool = True) -> Tuple[float, float, float]:

    """
    East, North, Up to target to geodetic coordinates

    Parameters
    ----------
    e : float
        East ENU coordinate (meters)
    n : float
        North ENU coordinate (meters)
    u : float
        Up ENU coordinate (meters)
    lat0 : float
           Observer geodetic latitude
    lon0 : float
           Observer geodetic longitude
    h0 : float
         observer altitude above geodetic ellipsoid (meters)
    ell : Ellipsoid
          wgs84 (default)
    deg : bool, optional
          degrees input/output  (False: radians in/out)


    Results
    -------
    lat : float
          geodetic latitude
    lon : float
          geodetic longitude
    alt : float
          altitude above ellipsoid  (meters)
    """

    x, y, z = enu2ecef(e, n, u, lat0, lon0, h0, deg=deg)

    return ecef2geodetic(x, y, z, deg=deg)

###enu2aer

In [ ]:
def enu2aer(e: float, n:float, u:float, deg: bool = True) -> Tuple:

    """
    ENU to Azimuth, Elevation, Range
    Parameters
    ----------
    e : float
        ENU East coordinate (meters)
    n : float
        ENU North coordinate (meters)
    u : float
        ENU Up coordinate (meters)
    deg : bool, optional
        degrees input/output  (False: radians in/out)
    Results
    -------
    azimuth : float
        azimuth to rarget
    elevation : float
        elevation to target
    srange : float
        slant range [meters]
    """

    # 1 millimeter precision for singularity stability

    TAU = 2 * np.pi

    try:
        e[abs(e) < 1e-3] = 0.0
        n[abs(n) < 1e-3] = 0.0
        u[abs(u) < 1e-3] = 0.0
    except TypeError:
        if abs(e) < 1e-3:
            e = 0.0  # type: ignore
        if abs(n) < 1e-3:
            n = 0.0  # type: ignore
        if abs(u) < 1e-3:
            u = 0.0  # type: ignore

    r = np.hypot(e, n)
    slantRange = np.hypot(r, u)
    elev = np.arctan2(u, r)
    az = np.arctan2(e, n) % TAU

    if deg:
        az = np.degrees(az)
        elev = np.degrees(elev)

    return az, elev, slantRange

###ecef2enu

In [ ]:
def ecef2enu(x: float, y: float, z: float, lat0: float, lon0: float, h0: float,
             deg: bool = True) -> Tuple[float, float, float]:

    """
    from observer to target, ECEF => ENU

    Parameters
    ----------
    x : target x ECEF coordinate (meters)
    y : target y ECEF coordinate (meters)
    z : target z ECEF coordinate (meters)
    lat0 : Observer geodetic latitude
    lon0 : Observer geodetic longitude
    h0 : observer altitude above geodetic ellipsoid (meters)
    ell : Ellipsoid, wgs84 (default)
    deg : (optional) degrees input/output  (False: radians in/out)

    Returns
    -------
    East : float
        target east ENU coordinate (meters)
    North : float
        target north ENU coordinate (meters)
    Up : float
        target up ENU coordinate (meters)

    """
    x0, y0, z0 = geodetic2ecef(lat0, lon0, h0, deg=deg)

    return uvw2enu(x - x0, y - y0, z - z0, lat0, lon0, deg=deg)

###ref2ecef

In [ ]:
def ref2ecef(ref: pd.DataFrame) -> pd.DataFrame:

    """
    Ajout d'une colonne à ref par conversion de la trajectoire de référence
    de coordonnées géodésique (az, el , alt) en géocentrique / ecef (x, y, z)
    """

    lat = ref.loc[:, 'lat'] * np.pi/180  # deg2rad
    lon = ref.loc[:, 'lon'] * np.pi/180  # deg2rad
    alt = ref.loc[:, 'alt']

    x, y, z = geodetic2ecef(lat.astype(float),
                            lon.astype(float),
                            alt.astype(float),
                            False)

    ref['x'] = x
    ref['y'] = y
    ref['z'] = z

    return ref